In [6]:
import tensorflow as tf
import numpy as np
import os

# from data_utils import *
#import classifiers.lstm as cell
from image_utils import load_image, preprocess_image, load_image_batch

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    return session

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# use only square images
face_test = preprocess_image(load_image('test_images/george.jpg', size=160))
face_batch = np.expand_dims(face_test, axis=0)
print(face_batch.shape)

(1, 160, 160, 3)


In [3]:
saver = tf.train.import_meta_graph('./pretrainned-model/model-20180402-114759.meta')
graph = tf.get_default_graph()

In [8]:
images_placeholder = graph.get_tensor_by_name("input:0")
embeddings = graph.get_tensor_by_name("embeddings:0")
phase_train_placeholder = graph.get_tensor_by_name("phase_train:0")
print(images_placeholder.shape)
print(embeddings.shape)


with get_session() as sess_test:
    saver.restore(sess_test, './pretrainned-model/model-20180402-114759.ckpt-275')
    # Run forward pass to calculate embeddings
    feed_dict = {
        images_placeholder: face_batch, 
        phase_train_placeholder:False 
    }
    emb = sess_test.run(embeddings, feed_dict=feed_dict)
    print(emb.shape)

(?, 160, 160, 3)
(?, 512)
INFO:tensorflow:Restoring parameters from ./pretrainned-model/model-20180402-114759.ckpt-275
(1, 512)


In [14]:
# use only square images
image_paths = [
    'test_images/george.jpg',
    'test_images/ebert.jpg',
    'test_images/ebert2.jpg',
    'test_images/ebert_dni.jpg'
]
images = load_image_batch(image_paths, size=160)
print(images.shape)

(4, 160, 160, 3)


In [15]:
with get_session() as sess_batch:
    saver.restore(sess_batch, './pretrainned-model/model-20180402-114759.ckpt-275')
    # Run forward pass to calculate embeddings
    feed_dict = {
        images_placeholder: images, 
        phase_train_placeholder:False 
    }
    img_embs = sess_batch.run(embeddings, feed_dict=feed_dict)
    
    print("1 with 2", np.linalg.norm(img_embs[0] - img_embs[1]))
    print("1 with 3", np.linalg.norm(img_embs[0] - img_embs[2]))
    print("1 with 4", np.linalg.norm(img_embs[0] - img_embs[3]))
    print("2 with 3", np.linalg.norm(img_embs[1] - img_embs[2]))
    print("2 with 4", np.linalg.norm(img_embs[1] - img_embs[3]))
    print("3 with 4", np.linalg.norm(img_embs[2] - img_embs[3]))
    print(emb.shape)
    """
    INFO:tensorflow:Restoring parameters from ./pretrainned-model/model-20180402-114759.ckpt-275
    1 with 2 1.1867632
    1 with 3 1.1089562
    1 with 4 1.3747356
    2 with 3 0.77131104
    2 with 4 1.1783515
    3 with 4 1.1392387
    (3, 512)
    
    At least 1 with 4 is greater than the other ones, need to fine tune the network for the correct images
    """

INFO:tensorflow:Restoring parameters from ./pretrainned-model/model-20180402-114759.ckpt-275
1 with 2 1.1867632
1 with 3 1.1089562
1 with 4 1.3747356
2 with 3 0.77131104
2 with 4 1.1783515
3 with 4 1.1392387
(3, 512)
